# Lecture 17 – Collision theory

In [ ]:
%pip install -q ampform plotly sympy

In [ ]:
from __future__ import annotations

from typing import Callable

import numpy as np
import plotly.graph_objects as go
import sympy as sp
from ampform.io import aslatex
from ampform.sympy import UnevaluatedExpression, implement_doit_method
from ampform.sympy.math import create_expression
from IPython.display import Math
from plotly.subplots import make_subplots

_This notebook is an attempt to recreate the Mathematica notebook [provided by Miguel Albaladejo](https://indico.ific.uv.es/event/6803/contributions/21224)._

## Riemann sheets

### Square root example

There are multiple solutions for $x$ to the equation $y^2 = x$ – the fact that we usually take $y = \sqrt{x}$ to be the solution to this equation is just a matter of convention. It would be more complete to represent the solution as a set of points in the complex plane. In this case, we have the set $S = \left\{\left(z, w\right)\in\mathbb{C}^2 | w^2=z\right\}$. This is set forms a **Riemann surface** in $\mathbb{C}^2$ space.

In [ ]:
def plot_riemann_surfaces(funcs: list[Callable], func_unicode: str) -> None:
    R, Θ = np.meshgrid(
        np.linspace(0, 1, num=30),
        np.linspace(-np.pi, +np.pi, num=60),
    )
    X = R * np.cos(Θ)
    Y = R * np.sin(Θ)
    Z = X + Y * 1j
    T = [f(Z) for f in funcs]
    vmax = max(max(T.real.max(), T.imag.max()) for T in T)

    style = dict(cmin=-vmax, cmax=+vmax, colorscale="RdBu_r")
    S_im = [
        go.Surface(x=X, y=Y, z=t.imag, surfacecolor=t.real, **style, name=f"S{i}")
        for i, t in enumerate(T, 1)
    ]
    S_re = [
        go.Surface(x=X, y=Y, z=t.real, surfacecolor=t.imag, **style, name=f"S{i}")
        for i, t in enumerate(T, 1)
    ]
    fig = make_subplots(
        cols=2,
        specs=[[{"type": "surface"}, {"type": "surface"}]],
        subplot_titles=(f"Im {func_unicode}", f"Re {func_unicode}"),
    )
    for i in range(len(funcs)):
        fig.add_trace(S_im[i], col=1, row=1)
        fig.add_trace(S_re[i], col=2, row=1)

    tv = [-1, 0, +1]
    tt = [f"{i:+d}" for i in tv]
    for i in (1, 2):
        fig.update_xaxes(tickvals=tv, ticktext=tt, title="Re(z)", col=i, row=1)
        fig.update_yaxes(tickvals=tv, ticktext=tt, title="Im(z)", col=i, row=1)
        fig.update_yaxes(tickvals=tv, ticktext=tt, title="", col=i, row=1)
    fig.update_layout(height=550, width=1_000)
    fig.update_traces(colorbar=dict(title="Re/Im"))
    fig.show()

In [ ]:
plot_riemann_surfaces(
    funcs=[lambda z: -np.sqrt(z), lambda z: +np.sqrt(z)],
    func_unicode="±√z",
)

A subset of this Riemann sheet is defined by the following function, which has only a single sheet structure for its real part.

In [ ]:
@implement_doit_method
class SignedSqrt(UnevaluatedExpression):
    is_commutative = True
    is_real = False

    def __new__(cls, z, sign, **hints) -> SignedSqrt:
        return create_expression(cls, z, sign, **hints)

    def evaluate(self) -> sp.Expr:
        z, sign = self.args
        return sp.sqrt(abs(z)) * sp.exp(sign * sp.I * sp.arg(z) / 2)

    def _latex(self, printer, *args) -> str:
        z = printer._print(self.args[0])
        sign = _render_sign(self.args[1], printer)
        return Rf"\sqrt[{sign}]{{{z}}}"


def _render_sign(sign, printer) -> str:
    if sign == +1:
        return "+"
    if sign == -1:
        return "-"
    return printer._print(sign)

In [ ]:
z = sp.Symbol("z")
neg_sqrt = SignedSqrt(z, sign=-1)
pos_sqrt = SignedSqrt(z, sign=+1)

In [ ]:
Math(aslatex({e: e.doit() for e in [neg_sqrt, pos_sqrt]}))

In [ ]:
plot_riemann_surfaces(
    funcs=[
        sp.lambdify(z, neg_sqrt.doit()),
        sp.lambdify(z, pos_sqrt.doit()),
    ],
    func_unicode="±√z",
)

<details>
<summary>Video explainers</summary>

<iframe width="560" height="315" src="https://www.youtube-nocookie.com/embed/R9MX8QgKwtg?si=kK-_1Po4XHzRpzR9" title="What are... Riemann surfaces? [VisualMath]" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture; web-share" allowfullscreen></iframe>

<iframe width="560" height="315" src="https://www.youtube.com/embed/sD0NjbwqlYw?si=XQMBxuwolPGlVbwq" title="Visualizing the Riemann zeta function and analytic continuation [3Blue1Brown]" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture; web-share" allowfullscreen></iframe>
  
</details>